# Imports, something is gonna throw an error, it's fine

In [1]:
from optparse import OptionParser
from collections import OrderedDict
import pickle
import os
import glob
import sys
import threading
import time

import PIL.Image
from IPython.display import clear_output, Image, display, HTML

import h5py
import numpy as np
import pandas as pd
import pysam
import tensorflow as tf

sys.path.append('../../basenji/')

from basenji import params
from basenji import seqnn
from basenji import vcf as bvcf

sys.path.append('../bin/')

from basenji_test import bigwig_open


UsageError: Invalid GUI request 'pdf', valid ones are:dict_keys(['qt5', 'ipympl', 'inline', 'widget', 'wx', 'qt', 'qt4', None, 'asyncio', 'osx', 'tk', 'notebook', 'nbagg', 'gtk', 'gtk3'])


# Specify local basenji repo

In [2]:
root_dir = '/n/local/basenji'

# Collect downloaded files and make 'job' object

In [3]:
params_file = os.path.join(root_dir,'manuscript/params.txt')
model_file = os.path.join(root_dir,'data/model.tf')
glob.glob(model_file+'*')

['/n/local/basenji/data/model.tf.meta',
 '/n/local/basenji/data/model.tf.index',
 '/n/local/basenji/data/model.tf.data-00000-of-00001']

In [4]:
job = params.read_job_params(params_file, require=['seq_length','num_targets'])

{'batch_buffer': 4096, 'cnn_dilation': [1, 1, 1, 1, 1, 1, 2, 4, 8, 16, 32, 64, 1], 'adam_beta1': 0.97, 'link': 'exp_linear', 'batch_size': 2, 'cnn_filters': [312, 368, 435, 514, 607, 717, 108, 108, 108, 108, 108, 108, 1365], 'loss': 'poisson', 'adam_beta2': 0.98, 'cnn_dropout': [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.05], 'num_targets': 4229, 'cnn_filter_sizes': [22, 1, 6, 6, 6, 3, 3, 3, 3, 3, 3, 3, 1], 'learning_rate': 0.002063, 'cnn_pool': [1, 2, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1], 'cnn_dense': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0], 'seq_length': 131072}


In [5]:
job['cnn_filter_sizes']
job['cnn_dilation']
job['cnn_pool']
job['cnn_filters']
job['cnn_dropout']
job['num_targets']


4229

# Import, build, and restore model

In [6]:
model = seqnn.SeqNN()

In [7]:
model.build_feed_sad(job)

INFO:tensorflow:Parsed params from job argument, and got {'cnn_l2_scale': 0.0, 'target_clip': None, 'cnn_dilation': [1, 1, 1, 1, 1, 1, 2, 4, 8, 16, 32, 64, 1], 'link': 'exp_linear', 'cnn_dense': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0], 'loss': 'poisson', 'batch_renorm_momentum': 0.9, 'grad_clip': 1.0, 'cnn_filter_sizes': [22, 1, 6, 6, 6, 3, 3, 3, 3, 3, 3, 3, 1], 'ensemble_during_prediction': False, 'batch_buffer': 4096, 'num_plateau_steps': 5000, 'adam_eps': 1e-08, 'batch_renorm': False, 'pool_by_unfolding': False, 'seq_length': 131072, 'augment_with_complement': False, 'cnn_filters': [312, 368, 435, 514, 607, 717, 108, 108, 108, 108, 108, 108, 1365], 'pool_by_averaging': False, 'cnn_dropout': [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.05], 'num_targets': 4229, 'learning_rate': 0.002063, 'batch_norm_momentum': 0.9, 'momentum': 0, 'ensemble_during_training': False, 'seq_depth': 4, 'learning_decay_rate': 0.2, 'target_pool': 1, 'adam_beta1': 0.97, 'final_l1_scale

In [8]:
saver = tf.train.Saver()
sess = tf.InteractiveSession()
saver.restore(sess, model_file)

INFO:tensorflow:Restoring parameters from /n/local/basenji/data/model.tf


# Collect and read through graph nodes

In [9]:
GraphNodes = OrderedDict([(n.name, n) for n in tf.get_default_graph().as_graph_def().node])

In [10]:
for op in GraphNodes.keys():
    print(op)

global_step/Initializer/zeros
global_step
global_step/Assign
global_step/read
inputs
targets
targets_na
is_training
zeros
stack
stack_1
map/TensorArray/size
map/TensorArray
map/TensorArray_1/size
map/TensorArray_1
map/TensorArrayUnstack/Shape
map/TensorArrayUnstack/strided_slice/stack
map/TensorArrayUnstack/strided_slice/stack_1
map/TensorArrayUnstack/strided_slice/stack_2
map/TensorArrayUnstack/strided_slice
map/TensorArrayUnstack/range/start
map/TensorArrayUnstack/range/delta
map/TensorArrayUnstack/range
map/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3
map/TensorArrayUnstack_1/Shape
map/TensorArrayUnstack_1/strided_slice/stack
map/TensorArrayUnstack_1/strided_slice/stack_1
map/TensorArrayUnstack_1/strided_slice/stack_2
map/TensorArrayUnstack_1/strided_slice
map/TensorArrayUnstack_1/range/start
map/TensorArrayUnstack_1/range/delta
map/TensorArrayUnstack_1/range
map/TensorArrayUnstack_1/TensorArrayScatter/TensorArrayScatterV3
map/Const
map/TensorArray_2/size
map/TensorArr

cnn9/batch_normalization/gamma
cnn9/batch_normalization/gamma/Assign
cnn9/batch_normalization/gamma/read
cnn9/batch_normalization/beta/Initializer/zeros
cnn9/batch_normalization/beta
cnn9/batch_normalization/beta/Assign
cnn9/batch_normalization/beta/read
cnn9/batch_normalization/moving_mean/Initializer/zeros
cnn9/batch_normalization/moving_mean
cnn9/batch_normalization/moving_mean/Assign
cnn9/batch_normalization/moving_mean/read
cnn9/batch_normalization/moving_variance/Initializer/ones
cnn9/batch_normalization/moving_variance
cnn9/batch_normalization/moving_variance/Assign
cnn9/batch_normalization/moving_variance/read
map/while/cnn9/batch_normalization/moments/mean/reduction_indices
map/while/cnn9/batch_normalization/moments/mean
map/while/cnn9/batch_normalization/moments/StopGradient
map/while/cnn9/batch_normalization/moments/SquaredDifference
map/while/cnn9/batch_normalization/moments/variance/reduction_indices
map/while/cnn9/batch_normalization/moments/variance
map/while/cnn9/batch_

In [11]:
GraphNodes['inputs']

name: "inputs"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
      dim {
        size: 131072
      }
      dim {
        size: 4
      }
    }
  }
}

In [12]:
GraphNodes['map/while/preds']

name: "map/while/preds"
op: "Select"
input: "map/while/Greater"
input: "map/while/add_1"
input: "map/while/Exp"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}

# Find variables in graph

In [13]:
tf.global_variables()

[<tf.Variable 'global_step:0' shape=() dtype=int64_ref>,
 <tf.Variable 'cnn0/conv1d/kernel:0' shape=(22, 4, 312) dtype=float32_ref>,
 <tf.Variable 'cnn0/batch_normalization/gamma:0' shape=(312,) dtype=float32_ref>,
 <tf.Variable 'cnn0/batch_normalization/beta:0' shape=(312,) dtype=float32_ref>,
 <tf.Variable 'cnn0/batch_normalization/moving_mean:0' shape=(312,) dtype=float32_ref>,
 <tf.Variable 'cnn0/batch_normalization/moving_variance:0' shape=(312,) dtype=float32_ref>,
 <tf.Variable 'cnn1/conv1d/kernel:0' shape=(1, 312, 368) dtype=float32_ref>,
 <tf.Variable 'cnn1/batch_normalization/gamma:0' shape=(368,) dtype=float32_ref>,
 <tf.Variable 'cnn1/batch_normalization/beta:0' shape=(368,) dtype=float32_ref>,
 <tf.Variable 'cnn1/batch_normalization/moving_mean:0' shape=(368,) dtype=float32_ref>,
 <tf.Variable 'cnn1/batch_normalization/moving_variance:0' shape=(368,) dtype=float32_ref>,
 <tf.Variable 'cnn2/conv1d/kernel:0' shape=(6, 368, 435) dtype=float32_ref>,
 <tf.Variable 'cnn2/batch_n

#### Get all variables except 'global_step' as numpy arrays. Dump to pickle file.

In [14]:
model_weights = OrderedDict([ (tensor.name, sess.run( tensor )) for tensor in tf.global_variables()[1:] ])

In [15]:
with open(os.path.join(root_dir,'manuscript/model_numpy.pkl'),'wb') as f:
    pickle.dump(model_weights, f)

#### Make sure the pickle loads okay.

In [16]:
with open(os.path.join(root_dir,'manuscript/model_numpy.pkl'),'rb') as f:
    check_weights = pickle.load(f)